# Getting stared
## Notebook initialisation
* Check MSTICPy package installed and latest version published
* Import all relevant packages
* Check configuration

In [ ]:
# pip install msticpy
# pip install msticnb

# Core MSTICPy initialisation for Notebooks

from msticpy.nbtools import nbinit
nbinit.init_notebook(namespace=globals())

## Query Provider initialisation
The code below is to initialise the query provider for Azure Sentinel

In [ ]:
from msticpy.data import QueryProvider
import pandas as pd

# Load query providers for Azure Sentinel
qry_prov_az = QueryProvider("AzureSentinel")

# Authentication
Connect the query provider to the targeted Workspace, the connection details are stored inside the msticpyconfig.yaml file

In [ ]:
# Get the Azure Sentinel workspace details from msticpyconfig
# Loading WorkspaceConfig with no parameters will use the details
# of your "Default" workspace (see the Configuring Azure Sentinel settings section earlier)
# If you want to connect to a specific workspace use this syntax:
#    ws_config = WorkspaceConfig(workspace="WorkspaceName")
# ('WorkspaceName' should be one of the workspaces defined in msticpyconfig.yaml)
ws_config = WorkspaceConfig()
 
# Connect to Azure Sentinel with our QueryProvider and config details
qry_prov_az.connect(ws_config)

To see all Workspaces configured in the msticpyconfig.yaml file loaded

In [ ]:
WorkspaceConfig.list_workspaces()

## Time parameters
nbwidgets.QueryTime is a class that allows user to filter the result set for the query parameters

In [ ]:
timerange = nbwidgets.QueryTime()
timerange

# Executing queries
## Built-in queries
Browse queries

In [ ]:
qry_prov_az.browse_queries()

Execute queries <br>
P.S. There are some built-in querties are built-in in the MSTICPy package, most of them require supplying parameters. In case, they are missing, the query cannot be executed and all required parameters are shown in the error message 

In [ ]:
qry_prov_az.WindowsSecurity.list_host_processes(timerange, host_name="*")

Extend built-in queries

In [ ]:
qry_prov_az.WindowsSecurity.list_host_processes(
    timerange,
    host_name="*",
    add_query_items="| where NewProcessName contains 'PowerShell'"
)

## Custom queries
Pass through native queries

In [ ]:
result = qry_prov_az.exec_query("prerecorded_CL | take 1000 | where TimeGenerated >= datetime('2021-09-01') | extend Event = tostring(parse_json(Message).EventID) | summarize NumberOfEvent = count(Event) by tostring(parse_json(Message).EventType)")
result.head(5)

# Define a query function
## Define a YAML file
YAML file template

YAML file example

In [ ]:
query_def = """
metadata:
    version: 1
    description: Security event queries
    data_environments: [AzureSentinel]
    data_families: [CustomQueries]
    tags: ['windows', 'event', 'security']
defaults:
    metadata: 
    parameters:
sources: 
    list_events_by_id:
        description: Retrieves list of events on a host
        args:
            query: '
                {table}
                | where EventID in ({event_list})
                | where TimeGenerated >= datetime({start})
                | where TimeGenerated <= datetime({end})
                {add_query_items}
            '
        parameters:
            table: (optional)
                description: The name of the source table
                type: str
                default: SecurityEvent
            start: (optional)
                description: start of the query
                type: datetime
                default: -1
            end: (optional)
                description: end of the query
                type: datetime
                default: 0
            event_list: (mandatory)
                description: List of event IDs to match
                type: list
            add_query_items: (optional)
                description: Additional query clauses (precede with "|")
                type: str
                default: ""
"""

## Query creation process
* Take a query from your query interface
* Parameterise the query
* Move any reusuable parameters to default section

In [ ]:
from pathlib import Path
if not Path("./queries").is_dir():
    Path("./queries").mkdir()

# Write out the yaml file
qry_file = Path("./queries").joinpath("az_sent_eventqueries.yaml")
qry_file.write_text(query_def)

# Check that the definition is readable and looks OK
from msticpy.data.data_query_reader import read_query_def_file
read_query_def_file(qry_file)

## Check the query

In [ ]:
from msticpy.data import QueryProvider
qry_prov_az = QueryProvider("AzureSentinel", query_paths=["./queries"])
